# Visualize GEDI Data using Lonboard

Authors: Harshini Girish (UAH), Rajat Shinde(UAH),Sheyenne Kirkland (UAH), Alex Mandel (DevSeed)

Date: 

Description: In the MAAP Algorithm Development Environment (ADE), Lonboard enables interactive geospatial visualization of GEDI calibration and validation data. By transforming CSV-based GEDI field measurements into a GeoDataFrame, we can map biomass and structural metrics across ecosystems.

## Run This Notebook

To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP’s ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## Additional Resources

 - [Visualizing STAC Items](https://docs.maap-project.org/en/latest/technical_tutorials/visualizing.html)  
  Official MAAP tutorial demonstrating how to visualize spatial footprints from STAC collections using different functions inside the ADE.

- [MAAP STAC Spatial Coverage Example ](https://github.com/MAAP-Project/maap-documentation-examples/blob/main/stac_spatial_coverage/notebooks/plot_items.ipynb)  
  A practical example from MAAP’s documentation repository showing how to retrieve and plot spatial items from a STAC catalog.


## Install/Import Packages

Make sure the following libraries are installed before running the notebook:

`!pip install lonboard pyogrio`


In [ ]:
import geopandas as gpd
import matplotlib as mpl
from lonboard import ScatterplotLayer, Map
from lonboard.colormap import apply_continuous_cmap
from shapely.geometry import Point
import pandas as pd
from pystac_client import Client
# import the MAAP package to handle queries
from maap.maap import MAAP

# import printing package to help display outputs
from pprint import pprint
import boto3

In [23]:
!pip install pystac-client


## Initializing the MAAP STAC Endpoint

Before beginning, we’ll form a connection to the MAAP STAC endpoint to set up

In [ ]:
maap = MAAP()
stac_endpoint = "https://stac.maap-project.org"
catalog = Client.open(stac_endpoint)


## Retrieving the Data


This code fetches and displays collections from a STAC endpoint. This code fetches and displays collections. It extracts id and title for each collection for further exploration or querying.

In [25]:
# Fetch all collections
collections = list(catalog.get_collections())
if collections:
    for col in collections:
        print(f"Collection ID: {col.id}")
        print(f"Title: {col.title}\n")
else:
    print("No collections found or error retrieving collections.")


Collection ID: ABoVE_UAVSAR_PALSAR
Title: Arctic-Boreal Vulnerability Experiment Uninhabited Aerial Vehicle Synthetic Aperture Radar Polarimetric SAR

Collection ID: AFRISAR_DLR
Title: AFRISAR_DLR

Collection ID: AFRISAR_DLR2
Title: AFRISAR_DLR2

Collection ID: AfriSAR_UAVSAR_Coreg_SLC
Title: AfriSAR UAVSAR Coregistered SLCs Generated Using NISAR Tools

Collection ID: AfriSAR_UAVSAR_Geocoded_Covariance
Title: AfriSAR UAVSAR Geocoded Covariance Matrix product Generated Using NISAR Tools

Collection ID: AfriSAR_UAVSAR_Geocoded_SLC
Title: AfriSAR UAVSAR Geocoded SLCs Generated Using NISAR Tools

Collection ID: AfriSAR_UAVSAR_KZ
Title: AfriSAR UAVSAR Vertical Wavenumber (KZ) Generated Using NISAR Tools

Collection ID: AfriSAR_UAVSAR_Normalization_Area
Title: AfriSAR UAVSAR Normalization Area Generated Using NISAR Tools

Collection ID: AfriSAR_UAVSAR_Ungeocoded_Covariance
Title: AfriSAR UAVSAR Ungeocoded Covariance Matrix product Generated Using NISAR Tools

Collection ID: BIOSAR1
Title: BI

In this notebook we use data set with collection with ID `GEDI_CalVal_Field_Data`, which contains GEDI calibration/validation field data. For each item, it extracts and stores the download URLs of all associated assets. 

In [50]:
collection_id = "GEDI_CalVal_Field_Data"

search = catalog.search(collections=[collection_id], max_items=5)
items = list(search.items())
asset_data = []
for item in items:
    for key, asset in item.assets.items():
        asset_data.append({
            "Item ID": item.id,
            "HREF": asset.href
        })
for entry in asset_data[:limit]:
    print(f"Item ID: {entry['Item ID']}")
    print(f"HREF: {entry['HREF']}")

Item ID: gedicalval_treedata_usa_neonyell_20200924_r03
HREF: s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Field_Data___2/gedicalval_treedata_usa_neonyell_20200924_r03.csv
Item ID: gedicalval_plotdata_usa_neonyell_20200924_r03
HREF: s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Field_Data___2/gedicalval_plotdata_usa_neonyell_20200924_r03.csv
Item ID: gedicalval_treedata_usa_neonpuum_20200924_r03
HREF: s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Field_Data___2/gedicalval_treedata_usa_neonpuum_20200924_r03.csv
Item ID: gedicalval_plotdata_usa_neonpuum_20200924_r03
HREF: s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Field_Data___2/gedicalval_plotdata_usa_neonpuum_20200924_r03.csv
Item ID: gedicalval_treedata_usa_neonwref_20200924_r03
HREF: s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Field_Data___2/gedicalval_treedata_usa_neonwref_20200924_r03.csv


This code initializes an S3 client to interact with NASA MAAP's cloud storage. Here the file is saved locally using the item's ID as the filename.

In [31]:
s3 = boto3.client('s3')
first_asset = asset_data[0]
s3_href = first_asset["HREF"]
bucket = "nasa-maap-data-store"
key = s3_href.split("s3://nasa-maap-data-store/")[1]
filename = f"{first_asset['Item ID']}.csv"

# Download the file
s3.download_file(bucket, key, filename)
print(f"Downloaded: {filename}")


Downloaded: gedicalval_treedata_usa_neonyell_20200924_r03.csv


## Load Data

Here the file is loaded into a pandas DataFrame named `df`. Each row in the CSV corresponds to a record (, and columns represent associated attributes.

In [51]:
csv_path = "gedicalval_treedata_usa_neonyell_20200924_r03.csv"
df = pd.read_csv(csv_path)


This step converts the `longitude` and `latitude` columns into spatial Point objects and wraps them in a GeoDataFrame (gdf).

In [52]:
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")


## Create and Display the Map

This step creates a `ScatterplotLayer`from the GeoDataFrame `gdf` and adds it to a Lonboard Map. The interactive map is rendered in the notebook, displaying all spatial points from your data.

In [89]:
layer = ScatterplotLayer.from_geopandas(gdf)
layer.get_fill_color = [0, 255, 0]
layer.radius_min_pixels = 4
m = Map(layers=[layer])
m


Map(layers=[ScatterplotLayer(get_fill_color=[0, 255, 0], radius_min_pixels=4.0, table=pyarrow.Table
project: s…

## Apply Color Styling Based on Biomass 

In [38]:
print(gdf.columns)


Index(['project', 'plot', 'subplot', 'survey', 'private', 'date', 'region',
       'vegetation', 'map', 'mat', 'pft.modis', 'pft.name', 'wwf.ecoregion',
       'latitude', 'longitude', 'p.sample', 'p.stemmap', 'p.origin',
       'p.orientation', 'p.shape', 'p.majoraxis', 'p.minoraxis', 'p.geom',
       'p.epsg', 'p.area', 'p.mindiam', 'sp.geom', 'sp.ix', 'sp.iy',
       'sp.shape', 'sp.area', 'sp.mindiam', 'pai', 'lai', 'cover', 'dft',
       'agb', 'agb.valid', 'agb.lower', 'agb.upper', 'agbd.ha',
       'agbd.ha.lower', 'agbd.ha.upper', 'sn', 'snd.ha', 'sba', 'sba.ha',
       'swsg.ba', 'h.t.max', 'sp.agb', 'sp.agb.valid', 'sp.agbd.ha',
       'sp.agbd.ha.lower', 'sp.agbd.ha.upper', 'sp.sba.ha', 'sp.swsg.ba',
       'sp.h.t.max', 'l.project', 'l.instr', 'l.epsg', 'l.date', 'g.fp',
       'tree.date', 'family', 'species', 'pft', 'wsg', 'wsg.sd', 'tree',
       'stem', 'x', 'y', 'z', 'status', 'allom.key', 'a.stem', 'h.t',
       'h.t.mod', 'd.stem', 'd.stem.valid', 'd.ht', 'c.w', 'm.a

This sets the color scale range by extracting the 10th and 90th percentile values of`m.agb`, helping to reduce the influence of outliers and highlight meaningful variation in the data.

In [77]:
q10 = gdf['m.agb'].quantile(0.1)
q90 = gdf['m.agb'].quantile(0.9)

The values of `m.abg` are scaled between 0 and 1 using `Normalize`, so they can be directly mapped onto a colormap. A reversed Viridis colormap is applied to the normalized data, generating RGB values for each point.

In [88]:
normalizer = mpl.colors.Normalize(q10, q90)
normalized_agb = normalizer(gdf['m.agb'])

viridis_colormap = mpl.colormaps["viridis_r"]
layer = ScatterplotLayer.from_geopandas(gdf)
layer.get_fill_color = viridis_colormap(normalized_agb, bytes=True)
layer.radius_min_pixels = 6

m2 = Map(layers=[layer])
m2


Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f4aac604dc0>
[
  [
  …